In [1]:
from __future__ import annotations

import json
from copy import deepcopy
from pathlib import Path

import numpy as np
import pandas as pd
import yaml

In [2]:
years = ["2016", "2017", "2018"]
years_vfp = ["2016APV", "2016", "2017", "2018"]

### Datasets

Data

In [ ]:
with open("samples/JetHT.yaml") as f:
    samples = yaml.safe_load(f)["samples"]

lines = []

for year in years:
    datasets = list(samples[f"JetHT{year}"]["datasets"].values())
    for i in range(len(datasets)):
        datasets[i] = datasets[i].replace("_", "\_")

    lend = len(datasets)
    lines.append(rf"\multirow{{{lend}}}{{*}}{{ {year} }} & {datasets[0]} \\" + "\n")

    for dataset in datasets[1:]:
        lines.append(rf" & {dataset} \\" + "\n")

    lines.append(r"\hline" + "\n")

# remove trailing "\\" and "\hline"
lines = lines[:-1]
lines[-1] = lines[-1][:-4]

with open("tables/datasets_jetht.tex", "w") as f:
    f.writelines(lines)

In [ ]:
with open("samples/SingleMuon.yaml") as f:
    samples = yaml.safe_load(f)["samples"]

lines = []

for year in years:
    datasets = list(samples[f"SingleMu{year}"]["datasets"].values())
    for i in range(len(datasets)):
        datasets[i] = datasets[i].replace("_", "\_")

    lend = len(datasets)
    lines.append(rf"\multirow{{{lend}}}{{*}}{{ {year} }} & {datasets[0]} \\" + "\n")

    for dataset in datasets[1:]:
        lines.append(rf" & {dataset} \\" + "\n")

    lines.append(r"\hline" + "\n")

# remove trailing "\\" and "\hline"
lines = lines[:-1]
lines[-1] = lines[-1][:-4]

with open("tables/datasets_singlemuon.tex", "w") as f:
    f.writelines(lines)

MC

In [ ]:
def process_xsec(xsec, round=3):
    if isinstance(xsec, str):
        xsec = eval(xsec)

    xsec = str(np.round(xsec, round))

    # remove trailing zeros
    for i in range(len(xsec)):
        if xsec[-1] == "0":
            xsec = xsec[:-1]
        else:
            break

    # remove trailing "."
    if xsec[-1] == ".":
        xsec = xsec[:-1]

    return xsec

In [ ]:
# process or sample and corresponding citation in paper
refs = {
    "WW": "Gehrmann:2014fva",
    "WZ": "Campbell:2011bn",
    "ZZ": "Cascioli:2014yka",
    "SingleTop": "SingleTopNNLORef",
    "TTbar": "TtbarNNLO",
    "HH": "LHCHiggsHH",
    # "HWW (inclusive)": "YR4",
    # "Hbb (inclusive)": "YR4",
    # "QCD": "xsdb",
    # "V+Jets": "xsdb"
}

In [ ]:
with open("samples/MC_bg.yaml") as f:
    samples = yaml.safe_load(f)["samples"]

with open("../data/xsecs.json") as f:
    xsecs = json.load(f)

lines = []

for sample in np.sort(list(samples.keys())):
    sample_text = sample
    if sample in refs:
        sample_text += rf"~\cite{{{refs[sample]}}}"

    for key, dataset in samples[sample]["datasets"].items():
        dname = dataset.split("/")[1].replace("_", "\_")
        citation = "" if key not in refs else rf"~\cite{{{refs[key]}}}"
        lines.append(rf" & {dname} & {process_xsec(xsecs[key])}{citation} \\" + "\n")

    lend = len(samples[sample]["datasets"])
    lines[-lend] = rf"\multirow{{{lend}}}{{*}}{{{sample_text}}}" + lines[-lend]
    lines.append(r"\hline" + "\n")

# remove trailing "\\" and "\hline"
lines = lines[:-1]
lines[-1] = lines[-1][:-4]

with open("tables/datasets_mcbg.tex", "w") as f:
    f.writelines(lines)

In [ ]:
with open("samples/MC_sig.yaml") as f:
    samples = yaml.safe_load(f)["samples"]

with open("../data/xsecs.json") as f:
    xsecs = json.load(f)

lines = []

for sample in np.sort(list(samples.keys())):
    for key, dataset in samples[sample]["datasets"].items():
        if key.startswith("GluGluToHHTo"):
            key = key.split("_pn4q")[0]
            cl = key.split("cHHH")[1].split("_")[0].replace("p", ".")
            sm = "SM" if cl == "1" else "BSM"
            pname = rf"ggHH {sm} ($\kappa_{{\lambda}} = {cl}$)"
        else:
            cv = key.split("CV_")[1].split("_C2V")[0].replace("_", ".")
            c2v = key.split("C2V_")[1].split("_C3")[0].replace("_", ".")
            c3 = key.split("C3_")[1].replace("_", ".")
            sm = "SM" if (cv == "1" and c2v == "1" and c3 == "1") else "BSM"
            pname = rf"VBFHH {sm} ($\kappa_\mathrm{{V}} = {cv}$, $\kappa_\mathrm{{2V}} = {c2v}$, $\kappa_{{\lambda}} = {c3}$)"

        dname = dataset.split("/")[1].replace("_", "\_")
        lines.append(
            rf"{pname} & {dname} & {float(process_xsec(xsecs[key], 10)) * 1000:.2f} \\" + "\n"
        )

# remove trailing "\\"
lines[-1] = lines[-1][:-4]

with open("tables/datasets_mcsig_nonres.tex", "w") as f:
    f.writelines(lines)

In [ ]:
def mxmy(sample):
    mY = int(sample.split("-")[-1])
    mX = int(sample.split("NMSSM_XToYHTo2W2BTo4Q2B_MX-")[1].split("_")[0])

    return (mX, mY)


with open("../data/xhy/parsed_miniaod_2017.yaml") as f:
    samples = yaml.safe_load(f)["samples"]["XHY"]["datasets"]

mps = [[*mxmy(sample)] for sample in samples]
mxs = np.sort(np.unique(np.array(mps)[:, 0]))
mys = np.sort(np.unique(np.array(mps)[:, 1]))

lines = []

line = r"$M_Y$ [\gev] & " + " & ".join(mxs.astype(str)) + r"\\" + "\n"
lines.append(line)

for my in mys:
    line = rf"{my}"
    for mx in mxs:
        line += " & "
        if [mx, my] in mps:
            line += r"$\checkmark$ "
    line += r"\\" + "\n"
    lines.append(line)

# remove trailing "\\\n"
lines[-1] = lines[-1][:-3]

with open("tables/datasets_mcsig_res.tex", "w") as f:
    f.writelines(lines)

### Triggers

In [ ]:
from triggers import HLTs

lines = []

for year in years:
    hlts = deepcopy(HLTs[year])
    for i in range(len(hlts)):
        hlts[i] = hlts[i].replace("_", "\_")

    lend = len(hlts)
    lines.append(rf"\multirow{{{lend}}}{{*}}{{ {year} }} & {hlts[0]} \\" + "\n")

    for hlt in hlts[1:]:
        lines.append(rf" & {hlt} \\" + "\n")

    lines.append(r"\hline" + "\n")

# remove trailing "\\" and "\hline"
lines = lines[:-1]
lines[-1] = lines[-1][:-4]

with open("tables/hlts.tex", "w") as f:
    f.writelines(lines)

In [ ]:
from triggers import muon_HLTs

lines = []

for year in years:
    hlts = deepcopy(muon_HLTs[year])
    for i in range(len(hlts)):
        hlts[i] = hlts[i].replace("_", "\_")

    lend = len(hlts)
    lines.append(rf"\multirow{{{lend}}}{{*}}{{ {year} }} & {hlts[0]} \\" + "\n")

    for hlt in hlts[1:]:
        lines.append(rf" & {hlt} \\" + "\n")

    lines.append(r"\hline" + "\n")

# remove trailing "\\" and "\hline"
lines = lines[:-1]
lines[-1] = lines[-1][:-4]

with open("tables/hlts_singlemuon.tex", "w") as f:
    f.writelines(lines)

## Lund plane SFs

Nonresonant

In [6]:
# templates_dir = Path("../src/HHbbVV/postprocessing/templates/24Apr26NonresBDT995AllSigs")
templates_dir = Path("../src/HHbbVV/postprocessing/templates/25Feb6NonresMatchingFix")
dfs = {
    "ggF": pd.read_csv(templates_dir / "lpsfs_passggf.csv"),
    "VBF": pd.read_csv(templates_dir / "lpsfs_passvbf.csv"),
}

sig_map = {
    "HHbbVV": r"SM ggF \HH",
    "VBFHHbbVV": r"SM VBF \HH",
    "qqHH_CV_1_C2V_0_kl_1_HHbbVV": r"VBF \HH ($\kapvv = 0$)",
    "qqHH_CV_1_C2V_2_kl_1_HHbbVV": r"VBF \HH ($\kapvv = 2$)",
}

df = dfs["ggF"]

lines = []

for j, (region, df) in enumerate(dfs.items()):
    sigs = df.to_numpy()[:, 0]
    for i, (sig, siglabel) in enumerate(sig_map.items()):
        if i == 0:
            region_label = rf"\multirow{{{len(sig_map)}}}{{*}}{{{region}}}"
        else:
            region_label = ""

        line = [region_label, siglabel]
        row = df[df["Unnamed: 0"] == sig]

        sf = row["SF"].values[0][:4]
        pm = row["SF"].values[0].split("+")[1].split("-")
        line.append(rf"${sf}^{{+{pm[0]}}}_{{-{pm[1]}}}$")

        for unc in ["stat_unc", "sys_unc", "sj_pt_unc", "np_unc"]:
            line.append(f"${row[unc].values[0]:.2f}$")

        for unc in ["unmatched", "dist"]:
            pm = row[unc].values[0].split("+")[1].split("-")
            line.append(rf"${{}}^{{+{pm[0]}}}_{{-{pm[1]}}}$")

        lines.append(" & ".join(line) + r" \\" + "\n")

    if j != len(dfs) - 1:
        lines[-1] = lines[-1][:-1] + r"[3mm]" + "\n"

# remove trailing "\\" and "\n"
lines[-1] = lines[-1][:-4]

with Path("tables/nonres_lpsfs.tex").open("w") as f:
    f.writelines(lines)

In [ ]:
row["SF"]

Resonant

In [ ]:
from HHbbVV.hh_vars import res_samples
from HHbbVV.postprocessing import utils

# pick random subset for table
table_samples = list(res_samples.values())[::16]
templates_dir = Path("/ceph/cms/store/user/rkansal/bbVV/templates/25Feb8XHYFix")


def _sig_map(sample: str):
    mx, my = utils.mxmy(sample)
    return rf"X[{mx}]$\rightarrow$HY[{my}]"


lines = []

for sample in table_samples:
    df = pd.read_csv(templates_dir / sample / f"lpsfs_pass.csv")
    line = [_sig_map(sample)]

    sf = df["SF"].values[0][:4]
    pm = df["SF"].values[0].split("+")[1].split("-")
    line.append(rf"${sf}^{{+{pm[0]}}}_{{-{pm[1]}}}$")

    for unc in ["stat_unc", "sys_unc", "sj_pt_unc", "np_unc"]:
        line.append(f"${df[unc].values[0]:.2f}$")

    for unc in ["unmatched", "dist"]:
        pm = df[unc].values[0].split("+")[1].split("-")
        line.append(rf"${{}}^{{+{pm[0]}}}_{{-{pm[1]}}}$")

    lines.append(" & ".join(line) + r" \\" + "\n")

# remove trailing "\\" and "\n"
lines[-1] = lines[-1][:-4]

with Path("tables/res_lpsfs.tex").open("w") as f:
    f.writelines(lines)